# QUDT Unit Conversion Tool

## A Website built out of a Jupyter notebook using Voila

In [6]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import requests
import json
import ipyvuetify as viewtify

my_headers = {'accept' : 'application/json'}

quant_response = requests.get("https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema%23> SELECT ?qk ?qkl WHERE { ?qk a qudt:QuantityKind . ?qk rdfs:label ?qkl . }", headers=my_headers)
qdata          = quant_response.json()
qresults       = qdata['results']
qbinds         = qresults['bindings']

def get_qvalues(obj):
    qvalues = []
    i       = 0

    for item in obj:
        elem = item['qk']
        qvalues.append(elem['value'])
        i+=1

    return qvalues

quant_values = get_qvalues(qbinds)

def get_qlabels(obj):
    qlabels = []
    i       = 0

    for item in obj:
        elem = item['qkl']
        qlabels.append(elem['value'])
        i+=1

    return qlabels

quant_labels = get_qlabels(qbinds)

file         = open("../images/qudt_logo-300x110.png", "rb")
image        = file.read()
label_style  = {'description_width' : 'initial'}
title_text   = viewtify.Html(tag = "div", children = ['Unit Conversion Tool'], style_ = " color: 'colors.indigo.darken4'; font-family: helvetica; font-size: 20px")
titleBox     = widgets.HBox([widgets.Image(value = image, format = 'png', width = 75, height = 27), title_text])
empty_label  = widgets.Text(value = '', description = '', disabled = True)
help_label   = widgets.HTML(value = '<a href = "../help/help.html"><b>Help</b></a>')
start_value  = widgets.Text(value = '1', description = 'Value to convert', style = label_style, disabled = False)

quant_dd = widgets.Dropdown(
    options     = quant_labels,
    description = 'Desired Quantity:',
    style       = label_style,
    disabled    = False,
)

def start_units_get_request(selected_quant):
    stunit_query_start = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX quantitykind: <http://qudt.org/schema/qudt/quantitykind/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema%23> SELECT ?stunit ?stunit_label WHERE { <"
    stunit_query_quant = quant_values[quant_dd.index]
    stunit_query_end   = "> qudt:applicableUnit ?stunit . ?stunit rdfs:label ?stunit_label . }"
    stunit_query       = stunit_query_start + selected_quant + stunit_query_end

    stunit_response = requests.get(stunit_query, headers = my_headers)
    studata         = stunit_response.json()
    sturesults      = studata['results']
    stubinds        = sturesults['bindings']

    return stubinds

def get_stuvalues(obj):
    stuvalues = []
    i         = 0

    for item in obj:
        elem = item['stunit']
        stuvalues.append(elem['value'])
        i += 1

    return stuvalues

def get_stulabels(obj):
    stulabels = []
    i         = 0

    for item in obj:
        elem = item['stunit_label']
        stulabels.append(elem['value'])
        i += 1

    return stulabels

stubinds      = start_units_get_request(quant_values[quant_dd.index])
stunit_values = get_stuvalues(stubinds)
stunit_labels = get_stulabels(stubinds)

start_unit_dd = widgets.Dropdown(
    options     = stunit_labels,
    description = 'Starting Unit:',
    style       = label_style,
    disabled    = False,
)

def target_units_get_request(selected_quant, stunits):
    tgtunit_query_start  = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/> PREFIX unit: <http://qudt.org/vocab/unit/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema%23> SELECT ?tgtunit ?tgtunit_label WHERE { <"
    unit_query_quant     = quant_values[quant_dd.index]
    tgtunit_query_middle = "> qudt:applicableUnit ?tgtunit . ?tgtunit rdfs:label ?tgtunit_label . FILTER (?tgtunit != <"
    stunit_value         = stunits[start_unit_dd.index]
    tgtunit_query_end    = ">) . }"
    tgtunit_query        = tgtunit_query_start + selected_quant + tgtunit_query_middle + stunit_value + tgtunit_query_end

    tgtunit_response = requests.get(tgtunit_query, headers = my_headers)
    tgtudata         = tgtunit_response.json()
    tgturesults      = tgtudata['results']
    tgtubinds        = tgturesults['bindings']

    return tgtubinds

def get_tgtuvalues(obj):
    tgtuvalues = []
    i          = 0

    for item in obj:
        elem = item['tgtunit']
        tgtuvalues.append(elem['value'])
        i += 1

    return tgtuvalues

def get_tgtulabels(obj):
    tgtulabels = []
    i          = 0

    for item in obj:
        elem = item['tgtunit_label']
        tgtulabels.append(elem['value'])
        i += 1

    return tgtulabels

tgtubinds      = target_units_get_request(quant_values[quant_dd.index], stunit_values)
tgtunit_values = get_tgtuvalues(tgtubinds)
tgtunit_labels = get_tgtulabels(tgtubinds)

target_unit_dd = widgets.Dropdown(
    options     = tgtunit_labels,
    description = 'Target Unit:',
    style       = label_style,
    disabled    = False,
)

def starting_unit_cm_get_request():
    stunitcm_query_start = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX unit: <http://qudt.org/vocab/unit/> SELECT DISTINCT ?stunitcm WHERE { <"
    stunitcm_query_unit  = stunit_values[start_unit_dd.index if start_unit_dd.index else 0]
    stunitcm_query_end   = "> qudt:conversionMultiplier ?stunitcm . }"
    stunitcm_query       = stunitcm_query_start + stunitcm_query_unit + stunitcm_query_end
    stunitcm_response    = requests.get(stunitcm_query, headers=my_headers)
    stucmdata            = stunitcm_response.json()
    stucmresults         = stucmdata['results']
    stucmbinds           = stucmresults['bindings']
    if len(stucmbinds) > 0:
        stucmelem         = stucmbinds[0]
        stucm             = stucmelem['stunitcm']
        stucmvalue        = stucm['value']
        start_unit_cm     = stucmvalue
    else:
        start_unit_cm = 1.0

    return start_unit_cm

def target_unit_cm_get_request():
    tgtunitcm_query_start = "https://qudt.org/fuseki/qudt/query?query=PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX unit: <http://qudt.org/vocab/unit/> SELECT DISTINCT ?tgtunitcm WHERE { <"
    tgtunitcm_query_unit  = tgtunit_values[target_unit_dd.index if target_unit_dd.index else 0]
    tgtunitcm_query_end   = "> qudt:conversionMultiplier ?tgtunitcm . }"
    tgtunitcm_query       = tgtunitcm_query_start + tgtunitcm_query_unit + tgtunitcm_query_end
    tgtunitcm_response    = requests.get(tgtunitcm_query, headers=my_headers)
    tgtucmdata            = tgtunitcm_response.json()
    tgtucmresults         = tgtucmdata['results']
    tgtucmbinds           = tgtucmresults['bindings']
    if len(tgtucmbinds) > 0:
        tgtucmelem         = tgtucmbinds[0]
        tgtucm             = tgtucmelem['tgtunitcm']
        tgtucmvalue        = tgtucm['value']
        target_unit_cm     = tgtucmvalue
    else:
        target_unit_cm = 1.0

    return target_unit_cm

start_unit_cm   = 1
target_unit_cm  = 1
converted_value = float(start_value.value) * float(start_unit_cm) / float(target_unit_cm)
target_value    = widgets.Text(value = str(converted_value), description = 'Converted value', style = label_style, disabled=True)

def update_options_from_selected_quantity(*args): # *args represent zero (case here) or more arguments.
    selected_quant         = quant_values[quant_dd.index if quant_dd.index else 0]
    stubinds               = start_units_get_request(selected_quant)
    stunit_values          = get_stuvalues(stubinds)
    stunit_labels          = get_stulabels(stubinds)
    start_unit_dd.options  = stunit_labels
    tgtubinds              = target_units_get_request(selected_quant, stunit_values)
    tgtunit_values         = get_tgtuvalues(tgtubinds)
    tgtunit_labels         = get_tgtulabels(tgtubinds)
    target_unit_dd.options = tgtunit_labels
    start_unit_cm          = starting_unit_cm_get_request()
    target_unit_cm         = target_unit_cm_get_request()
    converted_value        = float(start_value.value) * float(start_unit_cm) / float(target_unit_cm)
    target_value.value     = str(converted_value)

def on_change_q(change):
    selected_quant  = quant_values[quant_dd.index if quant_dd.index else 0]

    if change['type'] == 'change' and change['name'] == 'index':
        update_options_from_selected_quantity()

quant_dd.observe(on_change_q)

def update_options_from_selected_start_unit(*args): # *args represent zero (case here) or more arguments.
    selected_quant         = quant_values[quant_dd.index if quant_dd.index else 0]
    tgtubinds              = target_units_get_request(selected_quant, stunit_values)
    tgtunit_values         = get_tgtuvalues(tgtubinds)
    tgtunit_labels         = get_tgtulabels(tgtubinds)
    target_unit_dd.options = tgtunit_labels
    start_unit_cm          = starting_unit_cm_get_request() if starting_unit_cm_get_request() != 'None' else 1
    target_unit_cm         = target_unit_cm_get_request() if target_unit_cm_get_request() != 'None' else 1
    converted_value        = float(start_value.value) * float(start_unit_cm) / float(target_unit_cm)
    target_value.value     = str(converted_value)

def on_change_su(change):
    selected_quant  = quant_values[quant_dd.index if quant_dd.index else 0]

    if change['type'] == 'change' and change['name'] == 'index':
        update_options_from_selected_start_unit()

start_unit_dd.observe(on_change_su)

def update_calculations_from_selected_start_value(*args): # *args represent zero (case here) or more arguments.
    start_unit_cm      = starting_unit_cm_get_request() if starting_unit_cm_get_request() != 'None' else 1
    target_unit_cm     = target_unit_cm_get_request() if target_unit_cm_get_request() != 'None' else 1
    converted_value    = float(start_value.value) * float(start_unit_cm) / float(target_unit_cm)
    target_value.value = str(converted_value)

def on_change_suv(change):
    if change['type'] == 'change' and change['name'] == 'value':
        update_calculations_from_selected_start_value()

start_value.observe(on_change_suv)

dyn_grid_items  = [quant_dd, help_label, start_unit_dd, target_unit_dd, start_value, target_value]
dyn_box         = widgets.GridBox(dyn_grid_items, layout=widgets.Layout(grid_template_columns="repeat(2,325px)"))
app_box         = widgets.VBox([titleBox, dyn_box])
display(app_box)
